In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [2]:
tau_d = 0.01
L_s_n = 0.01
R_s_n = 0.01
K_p_n = L_s_n/tau_d
K_i_n = R_s_n/tau_d
params_dict = {'L_s':L_s_n,'R_s':R_s_n,
               'N_pp':2.0,'Phi':1.0,
              'K_p':K_p_n,'K_i':K_i_n}


u_ini_dict = {'tau_r_ref':0.0,'i_d_ref':0.0}  # for the initialization problem
u_run_dict = {'tau_r_ref':0.0,'i_d_ref':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['i_d','i_q','xi_i_d','xi_i_q']    # [inductor current, PI integrator]
y_ini_list = ['tau_r','omega_e','v_d','v_q','i_q_ref'] # for the initialization problem
y_run_list = ['tau_r','omega_e','v_d','v_q','i_q_ref'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
params_dict = {'R_w':0.316,'G':9.81,'M':1200.0,'K_w':1.0,'C_rr':0.03,'Rho':1.225,'S_f':2.13,'C_x':0.32,'K_sign':100}


u_ini_dict = {'beta':0.0}  # for the initialization problem
u_run_dict = {'beta':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['nu','x_pos']    # [inductor current, PI integrator]
y_ini_list = ['omega_r','snu'] # for the initialization problem
y_run_list = ['omega_r','snu'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [4]:
tau_w = tau_r/K_w
f_w = tau_w/R_w
f_b = G*M*sym.sin(beta)
f_d = 0.5*Rho*S_f*C_x*nu**2*snu
f_r = C_rr*G*M*snu
p_r = tau_r*omega_r

dnu = 1/M*(f_w - f_d - f_r - f_b)
dx_pos = nu - x_pos*1e-4

eq_snu = -snu + (1/(1+sym.exp(-nu*K_sign)))*2-1
eq_omega_r =-omega_r +  nu/(K_w*R_w)


f_mech_list = [dnu,dx_pos]
g_mech_list = [eq_snu,eq_omega_r]

x_mech_list = [ nu,  x_pos]
y_mech_list = [ snu,omega_r]

u_mech_dict = {'beta':0.0}

params_mech_dict = {'R_w':0.316,'G':9.81,'M':1200.0,'K_w':1.0,'C_rr':0.03,'Rho':1.225,'S_f':2.13,'C_x':0.32,'K_sign':100}



In [5]:
# pmsm:
di_d = 1/L_s * (  v_d + L_s * omega_e * i_q - R_s * i_d  )
di_q = 1/L_s * (  v_q - L_s * omega_e * i_d - omega_e * Phi - R_s * i_q  )
eq_tau_r = -tau_r + 3/2*Phi*N_pp*i_q
eq_omega_e = -omega_e + N_pp*omega_r

# ctrl:
epsilon_i_d =  i_d_ref - i_d
epsilon_i_q =  i_q_ref - i_q
u_d = K_p*epsilon_i_d + K_i*xi_i_d
u_q = K_p*epsilon_i_q + K_i*xi_i_q

eq_v_d = -u_d + v_d + L_s * omega_e * i_q 
eq_v_q = -u_q + v_q - L_s * omega_e * i_d -  omega_e * Phi 
dxi_i_d = epsilon_i_d
dxi_i_q = epsilon_i_q
eq_i_q_ref = - tau_r_ref + 3/2*Phi*N_pp*i_q_ref

f_pmsm_list = [di_d,di_q,dxi_i_d,dxi_i_q]
g_pmsm_list = [eq_tau_r,eq_omega_e,eq_v_d,eq_v_q,eq_i_q_ref]

x_pmsm_list = [ i_d, i_q, xi_i_d, xi_i_q]
y_pmsm_list = [ tau_r, omega_e, v_d, v_q, i_q_ref]

u_pmsm_dict = {'tau_r_ref':0.0,'i_d_ref':0.0}

params_pmsm_dict = {'L_s':L_s_n,'R_s':R_s_n,
               'N_pp':2.0,'Phi':1.0,
              'K_p':K_p_n,'K_i':K_i_n}

In [6]:
f_list = f_pmsm_list + f_mech_list
g_list = g_pmsm_list + g_mech_list
x_list = x_pmsm_list + x_mech_list
y_list = y_pmsm_list + y_mech_list
params_dict = params_pmsm_dict
params_dict.update(params_mech_dict)
u_dict = u_pmsm_dict
u_dict.update(u_mech_dict)
h_dict = {'u_d':u_d,'u_q':u_q,'tau_r_ref':tau_r_ref,'v_pk':(v_d**2+v_q**2)**0.5,
         'f_w':f_w,'f_d':f_d,'f_r':f_r,'tau_r':tau_r,'p_r':tau_r*omega_r,'nu_kmh':3.6*nu}

sys = {'name':f"pmsm_mech",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_dict,
       'u_ini_dict':u_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)


C:\Users\jmmau\anaconda3\lib\site-packages\sympy\matrices\repmatrix.py:98: SymPyDeprecationWarning: 

non-Expr objects in a Matrix has been deprecated since SymPy 1.9. Use
list of lists, TableForm or some other data structure instead. See
https://github.com/sympy/sympy/issues/21497 for more info.

  SymPyDeprecationWarning(


0.029957056045532227
0.03190112113952637
Compilation time: 4.47 s


In [11]:
omega_r

omega_r

In [12]:
u_dict

{'tau_r_ref': 0.0, 'i_d_ref': 0.0, 'beta': 0.0}